In [81]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import jaccard_score
import pandas as pd
import re
from tqdm import tqdm

# PlantSeg imports
import plantseg
from plantseg.core.zoo import ModelZoo
from plantseg.tasks import import_image_task, unet_prediction_task

# CellPose imports
from cellpose import models, io

import napari


from pathlib import Path
from plantseg.tasks import import_image_task, unet_prediction_task
from plantseg.core import PlantSegImage, ImageProperties, image
import cv2
from plantseg.core.zoo import ModelZoo

mz = ModelZoo(plantseg.PATH_MODEL_ZOO, plantseg.PATH_MODEL_ZOO_CUSTOM)
plantseg_model_names = mz.get_model_names()

from cellpose import models, utils, io
import matplotlib.pyplot as plt
import cellpose.models

cp_models = ["cyto3", "nuclei", "cyto2_cp3", "tissuenet_cp3", "livecell_cp3", "yeast_PhC_cp3", "yeast_BF_cp3", "bact_phase_cp3", "bact_fluor_cp3", "deepbacs_cp3", "cyto2", "cyto", "CPx", "transformer_cp3", "neurips_cellpose_default", "neurips_cellpose_transformer", "neurips_grayscale_cyto2", "CP", "CPx", "TN1", "TN2", "TN3", "LC1", "LC2", "LC3", "LC4"]

ps_models = [
    "generic_confocal_3D_unet",
    "generic_light_sheet_3D_unet",
    "confocal_3D_unet_ovules_ds1x",
    "confocal_3D_unet_ovules_ds2x",
    "confocal_3D_unet_ovules_ds3x",
    "confocal_2D_unet_ovules_ds2x",
    "lightsheet_3D_unet_root_ds1x",
    "lightsheet_3D_unet_root_ds2x",
    "lightsheet_3D_unet_root_ds3x",
    "lightsheet_2D_unet_root_ds1x",
    "lightsheet_3D_unet_root_nuclei_ds1x",
    "lightsheet_2D_unet_root_nuclei_ds1x",
    "confocal_2D_unet_sa_meristem_cells",
    "confocal_3D_unet_sa_meristem_cells",
    "lightsheet_3D_unet_mouse_embryo_cells",
    "confocal_3D_unet_mouse_embryo_nuclei",
    "PlantSeg_3Dnuc_platinum",
]

import warnings

# Suppress PyTorch warnings
warnings.filterwarnings("ignore", message="You are using `torch.load`", category=FutureWarning)

In [82]:
# viewer = napari.Viewer()

In [ ]:
# Define the function to construct the dataset DataFrame
def construct_dataset_dataframe(base_dir):
    columns = ["dataset_name", "sequence_name", "image_path", "mask", "gold_mask"] + cp_models + ps_models

    data = []

    for dataset_name in os.listdir(base_dir):
        dataset_path = os.path.join(base_dir, dataset_name)
        if not os.path.isdir(dataset_path):
            continue

        for sequence_name in os.listdir(dataset_path):
            sequence_path = os.path.join(dataset_path, sequence_name)
            if not os.path.isdir(sequence_path) or "_" in sequence_name:
                continue

            # Paths for masks and gold masks
            err_seg_path = os.path.join(dataset_path, f"{sequence_name}_ERR_SEG")
            gt_seg_path = os.path.join(dataset_path, f"{sequence_name}_GT", "SEG")

            for image_file in os.listdir(sequence_path):
                if image_file.endswith(".tif"):
                    image_path = os.path.join(sequence_path, image_file)

                    # Corresponding mask and gold mask paths
                    mask_path = os.path.join(err_seg_path, f"mask{image_file[1:]}")
                    mask_path = mask_path if os.path.exists(mask_path) else None

                    gold_mask_path = os.path.join(gt_seg_path, f"man_seg{image_file[1:]}")
                    gold_mask_path = gold_mask_path if os.path.exists(gold_mask_path) else None

                    # Append a row with default None values for models
                    data.append([dataset_name, sequence_name, image_path, mask_path, gold_mask_path] + [None] * (len(columns) - 5))

    return pd.DataFrame(data, columns=columns)


# Define the base directory (update this to your actual dataset path)
base_dir = "./datasets"

# Create the DataFrame
dataset_df = construct_dataset_dataframe(base_dir)

# Save or display the DataFrame
print(dataset_df.head())
# Optionally save to a CSV file
dataset_df.to_csv("dataset_summary.csv", index=False)

In [ ]:
dataset_df.head(3)

In [85]:
# Helper function for loading images
def load_image(file_path):
    return cv2.imread(file_path, cv2.IMREAD_UNCHANGED)


# # Function to perform segmentation using PlantSeg
# def segment_with_plantseg(image_path, model_name="lightsheet_2D_unet_root_ds1x", patch=(1, 64, 64), device="cuda"):
#     plantseg_image = import_image_task(input_path=Path(image_path), semantic_type="raw", stack_layout="YX")
#     predicted_images = unet_prediction_task(image=plantseg_image, model_name=model_name, patch=patch, device=device)
#     # Return the first predicted image
#     return predicted_images[0].get_data()


# # Function to perform segmentation using CellPose
# def segment_with_cellpose(image_path, model_type="cyto2", flow_threshold=0.4, cellprob_threshold=0):
#     image = io.imread(image_path)
#     model = models.Cellpose(gpu=True, model_type=model_type)
#     masks, flows, styles, diams = model.eval(image, diameter=None, channels=[0, 0], flow_threshold=flow_threshold, cellprob_threshold=cellprob_threshold)
#     return masks


# Function for calculating IoU with ground truth
def calculate_iou(pred, gt):
    pred_flat = pred.flatten()
    gt_flat = gt.flatten()
    return jaccard_score(gt_flat, pred_flat, average="binary")


def get_images_for_masks(gt_gold_paths, image_paths):
    subset_paths = []
    for p in gt_gold_paths:
        first_gt_gold_file = os.path.basename(p)
        # Extract digits from the filename
        digits = int(re.findall(r"\d+", first_gt_gold_file)[0])
        subset_paths.append(image_paths[digits])
    return subset_paths


# def segment(image_path, model_name):
#     if model_name in cp_models:
#         return segment_with_cellpose(image_path, model_type=model_name)
#     elif model_name in ps_models:
#         return segment_with_plantseg(image_path, model_name=model_name)
#     else:
#         raise ValueError(f"Unknown model name: {model_name}")


# def seg_with_CellPose(image_path, model_type="cyto2", diameter=None):
#     image = io.imread(image_path)
#     model = models.CellposeModel(gpu=True, model_type=model_type)
#     masks, flows, styles = model.eval(image, diameter=diameter, channels=[0, 0], flow_threshold=0.4, cellprob_threshold=0)
#     return masks


# def seg_with_PlantSeg(image_path, model_name="lightsheet_2D_unet_root_ds1x", patch=(1, 64, 64), device="cuda"):
#     plantseg_image = import_image_task(input_path=Path(image_path), semantic_type="raw", stack_layout="YX")
#     predicted_images = unet_prediction_task(image=plantseg_image, model_name=model_name, patch=patch, device=device)
#     return predicted_images[0].get_data()


def ins_to_sem(mask):
    # Convert instance segmentation mask to semantic mask
    mask = mask.astype(np.uint8)
    mask = mask * 255
    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    return mask


def calc_jaccard_score(pred, gt):
    pred_flat = pred.flatten()
    gt_flat = gt.flatten()
    return jaccard_score(gt_flat, pred_flat, average="micro")


def calc_jaccard_score_per_object(pred, gt):
    # Unique labels for reference objects (ground truth) and predicted objects
    gt_labels = np.unique(gt)
    pred_labels = np.unique(pred)

    # Remove background label (assumed to be 0)
    gt_labels = gt_labels[gt_labels > 0]
    pred_labels = pred_labels[pred_labels > 0]

    jaccard_scores = []

    for gt_label in gt_labels:
        # Extract pixels for the current reference object
        gt_object = gt == gt_label

        # Find the matching predicted object with maximum overlap
        best_iou = 0
        for pred_label in pred_labels:
            pred_object = pred == pred_label
            intersection = np.logical_and(gt_object, pred_object).sum()
            union = np.logical_or(gt_object, pred_object).sum()

            if union > 0:
                iou = intersection / union
                best_iou = max(best_iou, iou)

        # Append the best IoU for this ground truth object
        jaccard_scores.append(best_iou)

    # Mean IoU across all ground truth objects
    return np.mean(jaccard_scores) if jaccard_scores else 0

In [86]:
# for index, row in dataset_df.iterrows():

#     for model in cp_models + ps_models:
#         image_path = row["image_path"][0]  # Assuming you want to use the first image in the list
#         gt_path = row["gt_path"][0]  # Assuming you want to use the first ground truth in the list
#         gt_gold_path = row["gt_gold_path"][0]  # Assuming you want to use the first ground truth gold in the list

#         # Perform segmentation
#         segmented_image = segment(image_path, model_name=model)

#         # Convert instance segmentation to semantic segmentation
#         semantic_mask = ins_to_sem(segmented_image)

#         # Load ground truth
#         gt_image = load_image(gt_gold_path)
#         gt_semantic = ins_to_sem(gt_image)

#         # Calculate IoU score
#         iou_score = calc_jaccard_score_per_object(semantic_mask, gt_semantic)
#         print(f"Model: {model}, IoU Score: {iou_score}")

In [87]:
# Function to perform segmentation
def segment(image_path, model_name):
    twoD_ps_models = [model for model in ps_models if "2D" in model]

    if model_name in cp_models:
        model = models.CellposeModel(gpu=True, model_type=model_name)
        image = io.imread(image_path)
        masks, _, _ = model.eval(image, diameter=None, channels=[0, 0], flow_threshold=0.4, cellprob_threshold=0)
        return masks
    elif model_name in twoD_ps_models:
        plantseg_image = import_image_task(input_path=Path(image_path), semantic_type="raw", stack_layout="YX")
        predicted_images = unet_prediction_task(image=plantseg_image, model_name=model_name, model_id=None, patch=(1, 64, 64), device="cuda")
        return predicted_images[0].get_data()
    else:
        raise ValueError(f"Unknown model name: {model_name}")


# Function to calculate Jaccard score
def calculate_jaccard_score(predicted_mask, ground_truth_mask):
    # Ensure both masks are the same data type
    pred_flat = predicted_mask.flatten().astype(np.int32)
    gt_flat = ground_truth_mask.flatten().astype(np.int32)
    return jaccard_score(gt_flat, pred_flat, average="micro")


# Main processing loop
def process_dataset(df, models):
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing dataset", position=0):
        try:
            image_path = row["image_path"]
            mask_path = row["mask"] if pd.notna(row["mask"]) else row["gold_mask"]
            
            assert image_path is not None
            assert mask_path is not None
            
            if not mask_path or not os.path.exists(mask_path):
                continue

            ground_truth_mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)

            for model_name in tqdm(models, desc=f"Row {idx}", leave=False, position=1):
                if pd.notna(row.get(model_name)):
                    continue  # Skip if Jaccard is already calculated
                try:
                    predicted_mask = segment(image_path, model_name)
                    iou_score = calculate_jaccard_score(predicted_mask, ground_truth_mask)
                    df.at[idx, model_name] = iou_score
                except Exception as e:
                    print(f"Error processing model {model_name} for image {image_path}: {e}")
        except Exception as e:
            # print(f"Error processing row {idx} for model {model_name}: {e}")
            print(image_path)
            print(mask_path)

In [29]:
dataset_df = pd.read_csv("dataset_summary.csv")

In [ ]:
filter_df = dataset_df.dropna(subset=["mask", "gold_mask"], how="all")
sampled_df = filter_df.groupby(["dataset_name", "sequence_name"], group_keys=False).apply(lambda x: x.sample(n=min(5, len(x)), random_state=42)).reset_index(drop=True)

In [ ]:
the_models = [f for f in ps_models if "2D" in f]
the_models

In [89]:
process_dataset(sampled_df, the_models)


100%|██████████| 345/345 [00:01<00:00, 247.04it/s]


INFO: P [MainThread] 2025-01-28 01:40:46,062 plantseg.functionals.prediction.prediction - Zoo prediction: Running model from PlantSeg official zoo.
INFO: P [MainThread] 2025-01-28 01:40:46,089 plantseg.functionals.prediction.prediction - Computing theoretical minimum halo from model.
INFO: P [MainThread] 2025-01-28 01:40:46,090 plantseg.functionals.prediction.prediction - For raw in shape (700, 1100): set patch shape (1, 64, 64), set halo shape (0, 44, 44)
INFO: P [MainThread] 2025-01-28 01:40:46,098 plantseg.functionals.prediction.utils.array_predictor - Using batch size of 1 for prediction


/space/alireza/tseg/.conda/lib/python3.11/site-packages/plantseg/io/tiff.py:114: UserWarning: No metadata found.
  warnings.warn("No metadata found.")









100%|██████████| 345/345 [00:00<00:00, 404.17it/s]


INFO: P [MainThread] 2025-01-28 01:40:47,020 plantseg.functionals.prediction.prediction - Zoo prediction: Running model from PlantSeg official zoo.
INFO: P [MainThread] 2025-01-28 01:40:47,053 plantseg.functionals.prediction.prediction - Computing theoretical minimum halo from model.
INFO: P [MainThread] 2025-01-28 01:40:47,054 plantseg.functionals.prediction.prediction - For raw in shape (700, 1100): set patch shape (1, 64, 64), set halo shape (0, 44, 44)
INFO: P [MainThread] 2025-01-28 01:40:47,061 plantseg.functionals.prediction.utils.array_predictor - Using batch size of 1 for prediction


/space/alireza/tseg/.conda/lib/python3.11/site-packages/plantseg/io/tiff.py:114: UserWarning: No metadata found.
  warnings.warn("No metadata found.")









Processing dataset:  73%|███████▎  | 73/100 [06:04<02:03,  4.56s/it]

...






100%|██████████| 180/180 [00:00<00:00, 403.19it/s]


INFO: P [MainThread] 2025-01-28 01:42:15,817 plantseg.functionals.prediction.prediction - Zoo prediction: Running model from PlantSeg official zoo.


/space/alireza/tseg/.conda/lib/python3.11/site-packages/plantseg/io/tiff.py:114: UserWarning: No metadata found.
  warnings.warn("No metadata found.")


INFO: P [MainThread] 2025-01-28 01:42:15,850 plantseg.functionals.prediction.prediction - Computing theoretical minimum halo from model.
INFO: P [MainThread] 2025-01-28 01:42:15,851 plantseg.functionals.prediction.prediction - For raw in shape (576, 720): set patch shape (1, 64, 64), set halo shape (0, 44, 44)
INFO: P [MainThread] 2025-01-28 01:42:15,857 plantseg.functionals.prediction.utils.array_predictor - Using batch size of 1 for prediction







Processing dataset: 100%|██████████| 100/100 [07:33<00:00,  4.53s/it]


In [ ]:
sampled_df

In [90]:
sampled_df.to_csv("sample_summary.csv", index=True)

In [12]:
dataset_df.to_csv("dataset_summary.csv", index=False)

In [ ]:
image_path = sampled_df["image_path"][0]
image_path

In [ ]:
plantseg_image = import_image_task(input_path=Path(image_path), semantic_type="raw", stack_layout="YX")

In [ ]:
predicted_images = unet_prediction_task(image=plantseg_image, model_name=the_models[0], model_id=None, patch=(1, 64, 64), device="cuda")
# Return the first predicted image
plt.imshow(predicted_images[0].get_data())

In [ ]:
# Columns representing the models
model_columns = cp_models + ps_models

# Group by dataset and sequence, then calculate the mean IoU for each model
average_iou = (
    sampled_df.groupby(["dataset_name", "sequence_name"])[model_columns]
    .mean()
    .reset_index()
)

# Calculate overall average IoU across datasets
overall_average_iou = average_iou[model_columns].mean().to_dict()

average_iou, overall_average_iou

In [ ]:
overall_average_iou